In [1]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [2]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
except IOError:
        print("File not accessible")
        

Enter a class to grade (i.e. class1 for class1)
Successfully opened class2.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [52]:
#count = len([grade for grade in grades if grade > 5])
from itertools import count


def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
a = has26Values(26)
#print(a)
z = map(valueCheck,grades)
z = list(z)
#print(z)
t = map(has26Values,z)
x = list(t)
b = x.count(False)
print(b)


2


In [3]:
checklist = grades
grade_df = pd.DataFrame(grades)
def valueCheck(x)
    length = len(x)
    
    print(len)


for grade in grades: 
            value = grade.split(",")
            length = len(value)
            count = sum(,grade)
            map()
            print(count)
            https://thispointer.com/python-count-elements-in-a-list-that-satisfy-certain-conditions/
            
            #print(length)
            if length != 26:
                
                print("Invalid line of data:"+ (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)
                print(incorrect_data)



SyntaxError: invalid syntax (3197450681.py, line 3)

Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
>>> from collections import Counter
>>> z = ['blue', 'red', 'blue', 'yellow', 'blue', 'red']
>>> Counter(z)
Counter({'blue': 3, 'red': 2, 'yellow': 1})

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'